# Imports

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from pathlib import Path
from hbt_tools import plots as hp

# Functions

In [ ]:
def print_stats(df, col):
    data = df.iloc[:, col]
    max_val = data.max()
    min_val = data.min()
    std_val = data.std()

    name = df.columns[col]
    ul = len(name) * '-'
    print(f"{name}")
    print(ul)
    print(f"max: {max_val}")
    print(f"min: {min_val}")
    print(f"std: {std_val}\n")
#-------------------------------------------------------------------------------

def hist_flow(
    ts_flow: pd.DataFrame,
    cols: list,
    title,
    names = ['Venturi', 'Nivus'],
    ):
    fig = make_subplots(rows=1, cols=len(cols), shared_yaxes=True)

    for i, col in enumerate(cols):
        fig.add_histogram(x=ts_flow.iloc[:,col], row=1, col=i+1, name=names[i])

    fig.update_layout(title_text=title)
    
    return fig
#-------------------------------------------------------------------------------

def filter_ts(ts, date_from, date_to):
    if not date_to:
        ts = ts[(ts.index>=date_from)]
    elif not date_from:
        ts = ts[(ts.index<date_to)]
    else:
        ts = ts[(ts.index>=date_from) & (ts.index<date_to)]

    return ts
#-------------------------------------------------------------------------------

def abs_diff(df1, df2):
    f = 60 / 1000
    d_sum1 = df1.groupby(pd.Grouper(freq='d')).sum() * f
    d_sum2 = df2.groupby(pd.Grouper(freq='d')).sum() * f

    abs_diff = d_sum1 - d_sum2

    return abs_diff
#-------------------------------------------------------------------------------

def perc_diff(df1, df2):
    f = 60 / 1000
    d_sum1 = df1.groupby(pd.Grouper(freq='d')).sum() * f
    d_sum2 = df2.groupby(pd.Grouper(freq='d')).sum() * f

    perc_diff = (d_sum1 - d_sum2) / d_sum1 * 100

    return perc_diff

# Parameters

In [ ]:
path_data = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.14 Richterswil Beratung "
    r"Aussenbauwerke\05 Berechnungen Grundlagen\Zulauf Nivus-Venturi"
    r"\Daten_Nivus-Venturi.xlsx")
save_dir = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.14 Richterswil Beratung "
    r"Aussenbauwerke\05 Berechnungen Grundlagen\Zulauf Nivus-Venturi")

# Read Data

In [ ]:
data = pd.read_excel(path_data)
data.set_index(data.columns[0], inplace=True)

In [ ]:
# mask negative values
mask = data < 0
data[mask] = float('nan')

# mask unplausible values
mask = data > 150
data[mask] = float('nan')

# Plots

## Comparison Nivus / Venturi

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data.iloc[:,0], name='Venturi'))
fig.add_trace(go.Scatter(x=data.index, y=data.iloc[:,1], name='Nivus'))

fig.update_xaxes(title='Datum')
fig.update_yaxes(title='Durchfluss [l/s]')

fig.update_layout(
    title='Zulauf Berg',
    font=dict(size=25)
)

In [ ]:
save_path = save_dir / 'Zulauf-Berg.html'
fig.write_html(save_path)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data.iloc[:,2], name='Venturi'))
fig.add_trace(go.Scatter(x=data.index, y=data.iloc[:,3], name='Nivus'))

fig.update_xaxes(title='Datum')
fig.update_yaxes(title='Durchfluss [l/s]')

fig.update_layout(
    title='Zulauf See',
    font=dict(size=25)
)

In [ ]:
save_path = save_dir / 'Zulauf-See.html'
fig.write_html(save_path)

## Comparison Berg / See

In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
    subplot_titles=['Zulauf Venturi', 'Zulauf Nivus'])

fig.add_trace(go.Scatter(x=data.index, y=data.iloc[:,0], name='Venturi Berg'), 
    row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data.iloc[:,2], name='Venturi See'),
    row=1, col=1)

fig.add_trace(go.Scatter(x=data.index, y=data.iloc[:,1], name='Nivus Berg'),
    row=2, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data.iloc[:,3], name='Nivus See'),
    row=2, col=1)

fig.update_xaxes(title='Datum', row=2, col=1)
fig.update_yaxes(title='Durchfluss [l/s]')

fig.update_layout(
    font=dict(size=25)
)

fig.update_annotations(
    font_size=28
)

In [ ]:
save_path = save_dir / 'Zulauf_Venturi-Nivus.html'
fig.write_html(save_path)

## Comparison Flow / Deviation

In [ ]:
# Berg
diff_berg = pd.DataFrame(data.iloc[:,0] - data.iloc[:,1])
mask = diff_berg.abs() > 30
diff_berg[mask] = float('nan')

df = pd.DataFrame(data.iloc[:,0])

save_path = save_dir / 'Diff_Berg.html'
fig = hp.ply_2y([df], [diff_berg], names=['Durchfluss Venturi', 'Abweichung Venturi-Nivus'],
    xlabel='Zeit', ylabels=['Durchfluss [l/s]', 'Delta Durchfluss [l/s]'], title='Abweichung Berg',
    save_path=save_path)
fig

In [ ]:
# See
diff_see = pd.DataFrame(data.iloc[:,2] - data.iloc[:,3])
mask = diff_see.abs() > 30
diff_see[mask] = float('nan')

df = pd.DataFrame(data.iloc[:,2])

save_path = save_dir / 'Diff_See.html'
fig = hp.ply_2y([df], [diff_see], names=['Durchfluss Venturi', 'Abweichung Venturi-Nivus'],
    xlabel='Zeit', ylabels=['Durchfluss [l/s]', 'Delta Durchfluss [l/s]'], title='Abweichung See',
    save_path=save_path)
fig

## Scatter Plot

In [ ]:
# Berg
size = 900
fig = px.scatter(x=data.iloc[:,0], y=data.iloc[:,1])
fig.add_shape(type='line', x0=0, y0=0, x1=140, y1=140, line=dict(dash='dash'))

fig.update_xaxes(title='Venturi Berg [l/s]')
fig.update_yaxes(title='Nivus Berg [l/s]')

fig.update_layout(
    font=dict(size=25),
    autosize=False,
    width=size,
    height=size
)

fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)

In [ ]:
save_path = save_dir / 'Scatter_Berg.html'
fig.write_html(save_path)

In [ ]:
# See
size = 900
fig = px.scatter(x=data.iloc[:,2], y=data.iloc[:,3])
fig.add_shape(type='line', x0=0, y0=0, x1=140, y1=140, line=dict(dash='dash'))

fig.update_xaxes(title='Venturi See [l/s]')
fig.update_yaxes(title='Nivus See [l/s]')

fig.update_layout(
    font=dict(size=25),
    autosize=False,
    width=size,
    height=size
)

fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)

In [ ]:
save_path = save_dir / 'Scatter_See.html'
fig.write_html(save_path)

## Daily Totals Absolute

In [ ]:
day_sum = data.groupby(pd.Grouper(freq='d')).sum() * (60 / 1000) # m3
abs_diff_berg = abs_diff(data.iloc[:,0], data.iloc[:,1])
abs_diff_see = abs_diff(data.iloc[:,2], data.iloc[:,3])

In [ ]:
# Berg
fig = make_subplots(rows=3, cols=1, shared_xaxes=True)
fig.add_trace(go.Scatter(x=day_sum.index, y=day_sum.iloc[:,0], name='Venturi Berg'),
    row=1, col=1)
fig.add_trace(go.Scatter(x=day_sum.index, y=day_sum.iloc[:,1], name='Nivus Berg'),
    row=2, col=1)
fig.add_trace(go.Scatter(x=abs_diff_berg.index, y=abs_diff_berg.values, name='Abweichung (abs) Berg'),
    row=3, col=1)

fig.update_xaxes(title='Datum', row=3, col=1)
fig.update_yaxes(title='Tagessumme [m3]')
fig.update_yaxes(title='Abweichung [m3]', row=3, col=1)

fig.update_layout(
    font=dict(size=22)
)

save_path = save_dir / 'Tagessummen_Berg_abs.html'
fig.write_html(save_path)

In [ ]:
# See
fig = make_subplots(rows=3, cols=1, shared_xaxes=True)
fig.add_trace(go.Scatter(x=day_sum.index, y=day_sum.iloc[:,2], name='Venturi See'),
    row=1, col=1)
fig.add_trace(go.Scatter(x=day_sum.index, y=day_sum.iloc[:,3], name='Nivus See'),
    row=2, col=1)
fig.add_trace(go.Scatter(x=abs_diff_see.index, y=abs_diff_see.values, name='Abweichung (abs) See'),
    row=3, col=1)

fig.update_xaxes(title='Datum', row=3, col=1)
fig.update_yaxes(title='Tagessumme [m3]')
fig.update_yaxes(title='Abweichung [m3]', row=3, col=1)

fig.update_layout(
    font=dict(size=22)
)

save_path = save_dir / 'Tagessummen_See_abs.html'
fig.write_html(save_path)

## Daily Totals Relative

In [ ]:
rel_diff_berg = perc_diff(data.iloc[:,0], data.iloc[:,1])
rel_diff_see = perc_diff(data.iloc[:,2], data.iloc[:,3])

In [ ]:
# Berg
fig = make_subplots(rows=3, cols=1, shared_xaxes=True)
fig.add_trace(go.Scatter(x=day_sum.index, y=day_sum.iloc[:,0], name='Venturi Berg'),
    row=1, col=1)
fig.add_trace(go.Scatter(x=day_sum.index, y=day_sum.iloc[:,1], name='Nivus Berg'),
    row=2, col=1)
fig.add_trace(go.Scatter(x=rel_diff_berg.index, y=rel_diff_berg.values, name='Abweichung (rel) Berg'),
    row=3, col=1)

fig.update_xaxes(title='Datum', row=3, col=1)
fig.update_yaxes(title='Tagessumme [m3]')
fig.update_yaxes(title='Abweichung [%]', row=3, col=1)

fig.update_layout(
    font=dict(size=22)
)

save_path = save_dir / 'Tagessummen_Berg_rel.html'
fig.write_html(save_path)

In [ ]:
# See
fig = make_subplots(rows=3, cols=1, shared_xaxes=True)
fig.add_trace(go.Scatter(x=day_sum.index, y=day_sum.iloc[:,2], name='Venturi See'),
    row=1, col=1)
fig.add_trace(go.Scatter(x=day_sum.index, y=day_sum.iloc[:,3], name='Nivus See'),
    row=2, col=1)
fig.add_trace(go.Scatter(x=rel_diff_see.index, y=rel_diff_see.values, name='Abweichung (rel) See'),
    row=3, col=1)

fig.update_xaxes(title='Datum', row=3, col=1)
fig.update_yaxes(title='Tagessumme [m3]')
fig.update_yaxes(title='Abweichung [%]', row=3, col=1)

fig.update_layout(
    font=dict(size=22)
)

save_path = save_dir / 'Tagessummen_See_rel.html'
fig.write_html(save_path)

## Histrograms

In [ ]:
# Berg
fig = hist_flow(data, cols=[0, 1], title='Histogramm Berg')
fig.update_layout(
    font=dict(size=25)
)

In [ ]:
save_path = save_dir / 'Histogramm-Berg.html'
fig.write_html(save_path)

In [ ]:
# See
fig = hist_flow(data, cols=[2, 3], title='Histogramm See')
fig.update_layout(
    font=dict(size=25)
)

In [ ]:
save_path = save_dir / 'Histogramm-See.html'
fig.write_html(save_path)

# Stats

In [ ]:
# Gesamtsumme Berg / See
data.sum() * 60 / 1000

In [ ]:
for i in range(len(data.columns)):
    print_stats(data, i)

# Compute TW

In [ ]:
date_format = '%d.%m.%Y %H:%M'
date_from = pd.to_datetime('20.08.2023 00:00')
date_to = pd.to_datetime('24.08.2023 00:00')
delta_t = date_to - date_from

q_ara = pd.DataFrame(data.iloc[:,2] + data.iloc[:,3])
q_ara = filter_ts(q_ara, date_from, date_to)
q_tw = float(q_ara.sum() * 60 / delta_t.total_seconds())

print(f"QTW: {q_tw:0.2f} l/s")